In [1]:
#imports

import pandas as pd
import numpy as np
import re, nltk, spacy, gensim
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from gensim import corpora, models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

pd.set_option('display.max_colwidth', -1)

## Classifying Tweets to Categories
<a id="intertopic_map"></a>

## LDA - Intertopic Distance Map

In [2]:
airlines = pd.read_csv('./data/with_sentiment.csv')

In [3]:
# Convert to list
data = airlines.clean.tolist()

pprint(data[:1])

['thanks']


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['thanks']]


In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['thank', 'thank take care happy customer']


In [6]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [7]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

In [8]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.5811807773482005 %


In [20]:
# Define Search Param
search_params = {'n_components': [6,7,8], 'learning_decay': [.5,.6,.7]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [21]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 6}
Best Log Likelihood Score:  -76532.05401294075
Model Perplexity:  616.7586612315497


In [22]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0     -19.625448 -58.822124  1       1        21.414612
2      0.046458   28.123964  2       1        17.143744
4      47.397202 -0.043824   3       1        15.848033
5     -41.373238 -8.203395   4       1        15.733031
1      4.336849  -18.545933  5       1        15.146445
3      35.238354 -53.781784  6       1        14.714135, topic_info=    Category        Freq        Term       Total  loglift  logprob
181  Default  336.000000  customer    336.000000  30.0000  30.0000
641  Default  358.000000  service     358.000000  29.0000  29.0000
355  Default  373.000000  hour        373.000000  28.0000  28.0000
531  Default  357.000000  plane       357.000000  27.0000  27.0000
65   Default  324.000000  bag         324.000000  26.0000  26.0000
746  Default  165.000000  try         165.000000  25.0000  25.0000
31   Default  173.000000  american    173.000000  24.0000  24.0000
122  Default  166.000000  check       166.000000  23.0000  23.0000
349  Default  131.000000  home        131.000000  22.0000  22.0000
103  Default  276.000000  cancel      276.000000  21.0000  21.0000
720  Default  448.000000  thank       448.000000  20.0000  20.0000
774  Default  219.000000  wait        219.000000  19.0000  19.0000
726  Default  180.000000  ticket      180.000000  18.0000  18.0000
285  Default  372.000000  flight      372.000000  17.0000  17.0000
528  Default  150.000000  pilot       150.000000  16.0000  16.0000
630  Default  207.000000  seat        207.000000  15.0000  15.0000
322  Default  137.000000  good        137.000000  14.0000  14.0000
441  Default  104.000000  max         104.000000  13.0000  13.0000
728  Default  345.000000  time        345.000000  12.0000  12.0000
191  Default  417.000000  delay       417.000000  11.0000  11.0000
86   Default  100.000000  boee        100.000000  10.0000  10.0000
119  Default  153.000000  change      153.000000  9.0000   9.0000 
84   Default  143.000000  board       143.000000  8.0000   8.0000 
651  Default  102.000000  sit         102.000000  7.0000   7.0000 
292  Default  379.000000  fly         379.000000  6.0000   6.0000 
522  Default  132.000000  people      132.000000  5.0000   5.0000 
635  Default  116.000000  send        116.000000  4.0000   4.0000 
64   Default  212.000000  bad         212.000000  3.0000   3.0000 
576  Default  114.000000  really      114.000000  2.0000   2.0000 
624  Default  203.000000  say         203.000000  1.0000   1.0000 
..       ...         ...  ...                ...     ...      ... 
771  Topic6   14.837383   virgin      15.652867   1.8629  -5.7617 
545  Topic6   13.859298   present     14.674429   1.8592  -5.8299 
724  Topic6   13.859108   thursday    14.674444   1.8592  -5.8299 
747  Topic6   13.859071   turkish     14.674433   1.8592  -5.8299 
456  Topic6   62.799953   mile        66.503294   1.8591  -4.3189 
430  Topic6   13.844834   luck        14.674285   1.8582  -5.8309 
647  Topic6   12.881646   shock       13.696014   1.8551  -5.9030 
426  Topic6   38.310388   lounge      41.108757   1.8459  -4.8131 
31   Topic6   147.119004  american    173.865085  1.7493  -3.4676 
396  Topic6   36.870844   landing     41.109136   1.8076  -4.8514 
749  Topic6   32.591375   tweet       36.183896   1.8118  -4.9748 
322  Topic6   110.433220  good        137.467733  1.6974  -3.7544 
635  Topic6   91.741635   send        116.617160  1.6764  -3.9398 
98   Topic6   62.784124   business    78.979707   1.6869  -4.3191 
20   Topic6   26.269150   airway      31.344126   1.7397  -5.1904 
208  Topic6   25.530867   diego       30.256378   1.7465  -5.2189 
131  Topic6   58.796881   class       86.319852   1.5324  -4.3847 
720  Topic6   160.130327  thank       448.819394  0.8857  -3.3828 
64   Topic6   89.887881   bad         212.219689  1.0573  -3.9603 
196  Topic6   54.051270   delta       100.703971  1.2941  -4.4689 
704  Topic6   43.358838  

In [23]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,service,customer,delay,hour,cancel,time,wait,make,thank,bad,flightle,minute,late,help,luggage
Topic 1,seat,home,today,really,problem,new,delay,connection,flight,miss,free,far,know,fuck,stop
Topic 2,bag,fly,try,check,board,airport,say,pay,make,let,time,wrong,gate,crew,lose
Topic 3,thank,american,good,max,boee,fly,send,bad,follow,mile,business,company,class,flight,extend
Topic 4,thank,ticket,change,help,need,people,make,great,number,denver,work,travel,love,agent,email
Topic 5,plane,flight,hour,pilot,passenger,sit,say,tell,late,gate,air,attendant,ask,chicago,leave


## Doc2Vec

In [13]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(airlines['text'])]

#  building a model
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, dm =1)
#Infer vector for a new document:
vector = model.infer_vector(["system", "response"])
#vector

In [14]:
similar_doc = model.docvecs.most_similar(3,topn = 50)
similar_doc.sort(key= lambda x: x[1])
similar_doc

[(5428, 0.9644858837127686),
 (809, 0.9645789861679077),
 (3881, 0.9646088480949402),
 (3828, 0.9646247029304504),
 (4533, 0.9649201035499573),
 (5272, 0.9649715423583984),
 (4209, 0.96499103307724),
 (5620, 0.9653065800666809),
 (1039, 0.9654243588447571),
 (5458, 0.9654476642608643),
 (6052, 0.9654936194419861),
 (5359, 0.9655624628067017),
 (4617, 0.9656705856323242),
 (5982, 0.9657090902328491),
 (5836, 0.9658951163291931),
 (4965, 0.9660129547119141),
 (3849, 0.9663909673690796),
 (1690, 0.9664311408996582),
 (3329, 0.9664399027824402),
 (3115, 0.966863751411438),
 (5006, 0.9674529433250427),
 (4548, 0.9675529599189758),
 (5286, 0.9676083326339722),
 (6090, 0.9676988124847412),
 (5528, 0.9677085280418396),
 (164, 0.9680057168006897),
 (5216, 0.9683314561843872),
 (4551, 0.9685839414596558),
 (3716, 0.9686228036880493),
 (4626, 0.9691300392150879),
 (4643, 0.9694578647613525),
 (4295, 0.9695259928703308),
 (4725, 0.9697233438491821),
 (6316, 0.9698335528373718),
 (5665, 0.970071315

In [15]:
airlines.iloc[[i[0] for i in similar_doc],:]

,Unnamed: 0,text,tweet_coord,tweet_created,tweet_location,negative,positive,neutral,compound,lemmatized,clean,blob_polarity,blob_subjectivity
5428,5428,IM ON UNITED AIRLINES AND THEY JUST PLAYED A SPIDERMAN FAR FROM HOME TRAILER IM SCREAMING,NaN,Wed Jun 19 20:43:29 +0000 2019,NaN,0.131,0.263,0.606,0.3818,"['played', 'spiderman', 'far', 'home', 'trailer', 'screaming']",played spiderman far home trailer screaming,0.100000,1.000000
809,809,Will definitely do so Sending a DM now,NaN,2015-02-23 04:25:00 -0800,USA / The World,0.000,0.310,0.690,0.4019,"['definitely', 'sending', 'dm']",definitely sending dm,0.000000,0.500000
3881,3881,Project Coordinator Regional Talent drawing the winners of our final Spring membership drive raffl,NaN,Wed Jul 03 21:00:13 +0000 2019,St. Louis,0.000,0.330,0.670,0.7096,"['project', 'coordinator', 'regional', 'talent', 'drawing', 'winner', 'final', 'spring', 'membership', 'drive', 'raffl']",project coordinator regional talent drawing winner final spring membership drive raffl,0.000000,1.000000
3828,3828,ICYMI United Doubles Flights Between San Francisco And Hong Kong,NaN,Sun Jul 07 22:00:00 +0000 2019,Your local newsstand,0.000,0.237,0.763,0.4215,"['icymi', 'double', 'flight', 'san', 'francisco', 'hong', 'kong']",icymi double flight san francisco hong kong,0.000000,0.000000
4533,4533,American Airlines and United Airlines Failed To Hit Their Targets They Reacted Very Differently,NaN,Fri Jul 05 23:01:51 +0000 2019,NaN,0.182,0.155,0.663,-0.1280,"['american', 'failed', 'hit', 'target', 'reacted', 'differently']",american failed hit target reacted differently,-0.166667,0.300000
5272,5272,SHARE VISIT SPAIN WITH THE UNITED AIRLINES IN LOS ANGELES CALIFORNIA UNITED STATES,NaN,Mon Jul 15 21:48:04 +0000 2019,🇻🇦🇵🇹🇪🇺🇳🇴,0.000,0.438,0.562,0.7783,"['share', 'visit', 'spain', 'los', 'angeles', 'california', 'state']",share visit spain los angeles california state,0.000000,0.000000
4209,4209,A380861 Reg A6EOM United for Wildlife Livery arrives and departs on June 30th 2019,NaN,Mon Jul 01 22:44:59 +0000 2019,"Los Angeles, CA",0.000,0.177,0.823,0.4215,"['reg', 'aeom', 'wildlife', 'livery', 'arrives', 'departs', 'june', 'th']",reg aeom wildlife livery arrives departs june th,0.000000,0.000000
5620,5620,People at Newark need help UNITED AIRLINES TREAT ALL THEIR COSTUMERS LIKE GARBAGE No consideration Send more people to wor,NaN,Thu Jul 18 23:23:00 +0000 2019,"San Antonio, Texas",0.073,0.429,0.498,0.8933,"['people', 'newark', 'need', 'help', 'treat', 'costumer', 'like', 'garbage', 'consideration', 'send', 'people', 'wor']",people newark need help treat costumer like garbage consideration send people wor,0.000000,0.000000
1039,1039,SO to for coming in clutch and finally taking me home,NaN,2015-02-22 17:19:59 -0800,NaN,0.000,0.000,1.000,0.0000,"['coming', 'clutch', 'finally', 'taking', 'home']",coming clutch finally taking home,0.000000,1.000000
5458,5458,Join w of Sedgwick amp of 812 at 10AM as they host the WCI2019 In,NaN,Mon Jul 22 23:44:22 +0000 2019,NaN,0.000,0.145,0.855,0.2960,"['join', 'w', 'sedgwick', 'host', 'wci']",join w sedgwick host wci,0.000000,0.000000
